In [ ]:
import os
import seaborn as sns
import comparators.transport as transport
import matplotlib.pyplot as plt
from data.parameters_names import ParametersNames as Parameters
from data.grid_configuration import CanonicalCoordinatesGridConfiguration
from transporters.madx.ptc_track.configuration import PtcTrackConfiguration
from transporters.approximator.configuration import ApproximatorConfiguration

In [ ]:
sns.set_style("whitegrid")

In [ ]:
path_to_optics = "path_to_folder_with_optic"

xml_file_name = "configuration_reco_beam1.xml"
approximator_number_in_xml = 2

serialized_approximator_file_name = "parametrization_6500GeV_0p4_185_reco.root"
approximator_name = "ip5_to_station_220_h_2_lhcb1"

In [ ]:
path_to_xml_file = os.path.join(path_to_optics, xml_file_name)
ptc_track_configuration = PtcTrackConfiguration.get_track_configuration_from_xml_file(path_to_xml_file, approximator_number_in_xml)

In [ ]:
path_to_approximator = os.path.join(path_to_optics, serialized_approximator_file_name)
approximator_configuration = ApproximatorConfiguration(path_to_approximator, approximator_name)

In [ ]:
x_mean = 0.0
delta_x = 4e-3
theta_x_mean = 0.000
delta_theta_x = 1.3e-3
y_mean = 0.0
delta_y = 4e-3
theta_y_mean = 0.00
delta_theta_y = 1.3e-3
pt_mean = -0.15
delta_pt = 0.15

number_of_x_values = 1
number_of_theta_x_values = 1
number_of_y_values = 1
number_of_theta_y_values = 1
number_of_pt_values = 12000

grid_configuration = CanonicalCoordinatesGridConfiguration.from_delta(x_mean, delta_x, number_of_x_values,
                                                        theta_x_mean, delta_theta_x, number_of_theta_x_values,
                                                        y_mean, delta_y, number_of_y_values,
                                                        theta_y_mean, delta_theta_y, number_of_theta_y_values,
                                                        pt_mean, delta_pt, number_of_pt_values)
particles = grid_configuration.generate_randomly()

In [ ]:
transporter1 = "ptc_track"
transporter2 = "approximator"

In [ ]:
transporters = {
    transporter1: ptc_track_configuration,
    transporter2: approximator_configuration
}

In [ ]:
datasets = transport.compare(particles, transporters)

In [ ]:
f, axes = plt.subplots(2, 2)

fig = plt.gcf()
fig.set_size_inches(40,20)

title_sufix = r"Error over training scope"

transport.plot(datasets, Parameters.X, Parameters.PT, title_sufix=title_sufix, ax=axes[0][0])
transport.plot(datasets, Parameters.THETA_X, Parameters.PT, title_sufix=title_sufix, ax=axes[0][1])
transport.plot(datasets, Parameters.Y, Parameters.PT, title_sufix=title_sufix, ax=axes[1][0])
transport.plot(datasets, Parameters.THETA_Y, Parameters.PT, title_sufix=title_sufix, ax=axes[1][1])